# HW1_Сравнительный анализ моделей классификации отзывов

**ФИО Студента:**

**Дата Выполнения:**

-----

## **1. Подготовка данных и EDA**

### **1.1. Загрузка и подготовка библиотек**

Начнем с установки и импорта необходимых библиотек для нашего проекта.

In [ ]:
# Установка библиотек
!pip install pymorphy3
!pip install gensim
!pip install lime

In [ ]:
# Основные библиотеки
import os
import re
import json
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from collections import Counter
import pickle
from datetime import datetime

# NLP библиотеки
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pymorphy3

# Embeddings
from gensim.models import Word2Vec, FastText
from gensim.models import KeyedVectors
import gensim.downloader as api

# ML
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                           f1_score, confusion_matrix, classification_report,
                           roc_auc_score, roc_curve)

# Визуализация
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import umap
import plotly.graph_objects as go
import plotly.express as px
from wordcloud import WordCloud

# Интерпретируемость
import shap
import lime

# Настройки
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)

# Создание структуры папок
folders = ['data', 'models', 'results', 'visualizations', 'reports']
for folder in folders:
    os.makedirs(folder, exist_ok=True)

# Загрузка ресурсов NLTK
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
print("NLTK ресурсы загружены")

# Инициализация морфологического анализатора и стоп-слов
morph = pymorphy3.MorphAnalyzer()
stop_words_ru = set(stopwords.words('russian'))
stop_words_en = set(stopwords.words('english'))

### **1.2. Загрузка и первичный анализ данных**

Загрузим датасет и проведем базовый анализ, чтобы понять его структуру.

In [ ]:
# Загрузка датасета
from datasets import load_dataset
df = load_dataset("d0rj/geo-reviews-dataset-2023")
df = pd.DataFrame(df['train'])

# ЗАДАНИЕ: Удалите нейтральные отзывы (рейтинг 3) и создайте бинарную целевую переменную 'sentiment'.
# 1 - позитивный отзыв (рейтинг 4, 5)
# 0 - негативный отзыв (рейтинг 1, 2)
# Удалите строки с отсутствующими значениями в 'sentiment'.



# ЗАДАНИЕ: Сбалансируйте классы, чтобы количество позитивных и негативных отзывов было одинаковым.



# Вывод базовой информации о датасете
print("Исследовательский анализ данных:")
print(f"\nРазмер датасета: {df.shape}")
print(f"\nТипы данных:\n{df.dtypes}")
print(f"\nПропущенные значения:\n{df.isnull().sum()}")

# Распределение классов
class_distribution = df['sentiment'].value_counts()
print("\nРаспределение классов:")
print(f"Позитивные (1): {class_distribution.get(1, 0)} ({class_distribution.get(1, 0)/len(df)*100:.1f}%)")
print(f"Негативные (0): {class_distribution.get(0, 0)} ({class_distribution.get(0, 0)/len(df)*100:.1f}%)")

### **1.3. Исследовательский анализ данных (EDA)**

Проведем более глубокий анализ текстовых данных и визуализируем результаты.

In [ ]:
# ЗАДАНИЕ: Добавьте в датафрейм столбцы с длиной текста, количеством слов и предложений.
# df['text_length'] = ...
# df['word_count'] = ...
# df['sentence_count'] = ...

# Вывод статистик по текстам
print("\nСтатистика текстов:")
print(df[['text_length', 'word_count', 'sentence_count']].describe())

# ЗАДАНИЕ: Создайте 6 графиков для визуализации EDA, например такие:
# 1. Распределение классов
# 2. Распределение количества слов
# 3. Boxplot количества слов по классам
# 4. Распределение количества предложений
# 5. Корреляция длины и sentiment
# 6. Средняя длина слова


-----

## **2. Предобработка текста**

На этом этапе мы создадим функцию для полной предобработки текстовых данных.  

- Напишите функцию, выполняющую лемматизацию, удаление стоп-слов, знаков препинания и приведение к нижнему регистру.  
- Примените функцию для создания колонки с обработанным текстом.   
- Проведите частотный анализ слов и визуализируйте облака слов для позитивных и негативных классов.  
- Разделите данные на обучающую, валидационную и тестовую выборки.  


In [ ]:
def preprocess_text(text,
                    language='ru',
                    use_lemmatization=True,
                    remove_stopwords=True,
                    min_word_length=2):
    """
    Полный pipeline предобработки текста.
    """
    # ЗАДАНИЕ: Реализуйте шаги предобработки:
    # 1. Приведение к нижнему регистру
    # 2. Удаление HTML и URLs
    # 3. Удаление спецсимволов
    # 4. Удаление лишних пробелов
    # 5. Токенизация
    # 6. Удаление стоп-слов
    # 7. Лемматизация
    # 8. Фильтрация по длине слова

    return ' '.join(tokens)

# ЗАДАНИЕ: Примените вашу функцию к столбцу 'text' и создайте новый столбец 'processed_text'.
# df['processed_text'] = ...

# Анализ результатов предобработки
print("\nЭффект предобработки:")
print(f"Средняя длина до обработки: {df['word_count'].mean():.1f} слов")
print(f"После полной обработки: {df['processed_text'].str.split().str.len().mean():.1f} слов")

# ЗАДАНИЕ: Проведите частотный анализ слов и создайте облака слов для позитивных и негативных отзывов.


-----

## **3. TF-IDF + LogisticRegression**

Векторизуем текст с помощью TF-IDF и обучим модель логистической регрессии.

- Подберите оптимальные параметры TfidfVectorizer (например, max_features, ngram_range), оценивая F1-score на валидационной выборке.  
- Обучите LogisticRegression на лучших TF-IDF признаках.  
- Оцените итоговое качество на тестовой выборке. Выведите отчет с метриками и confusion matrix.  
- Проанализируйте важность признаков (коэффициенты модели).  


In [ ]:
# Разделение данных на обучающую, валидационную и тестовую выборки
X = df['processed_text']
y = df['sentiment']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# ЗАДАНИЕ: Проведите эксперименты с различными параметрами TfidfVectorizer (max_features, ngram_range и т.д.).
# Обучите LogisticRegression на каждой конфигурации и выберите лучшую по метрике F1 на валидационной выборке.
# tfidf_params = [...]
# for params in tfidf_params:
#     ...

# Финальная оценка лучшей модели на тестовой выборке
# best_vectorizer = ...
# best_lr_model = ...
# X_test_tfidf = best_vectorizer.transform(X_test)
# y_test_pred_tfidf = best_lr_model.predict(X_test_tfidf)

# ЗАДАНИЕ: Рассчитайте и выведите метрики (Accuracy, Precision, Recall, F1, ROC_AUC) и confusion matrix.
# ...

# Анализ важности признаков
# ...

# Сохранение лучшей модели
# with open('models/tfidf_lr.pkl', 'wb') as f:
#     pickle.dump(best_lr_model, f)

-----

## **4. Word2Vec embeddings**

Теперь используем Word2Vec для получения векторных представлений текста.

- Обучите собственную модель Word2Vec на обучающей выборке. Подберите оптимальные параметры (vector_size, window, sg), оценивая F1-score классификатора на валидации.  
- Реализуйте функцию получения вектора документа путем усреднения векторов слов.
- Обучите LogisticRegression на полученных векторах.
- Оцените качество на тестовой выборке.

In [ ]:
# Подготовка данных для Word2Vec
tokenized_texts = [text.split() for text in df['processed_text']]

# ЗАДАНИЕ: Проведите эксперименты с параметрами Word2Vec (vector_size, window, sg).
# Для каждого набора параметров векторизуйте тексты (усредняя векторы слов) и обучите LogisticRegression.
# Выберите лучшую модель по F1 на валидации.
# w2v_params = [...]
# for params in w2v_params:
#     ...

# Финальная оценка лучшей модели
# ...

# ЗАДАНИЕ: Рассчитайте метрики и выведите confusion matrix.
# ...

# Анализ семантических отношений (найти похожие слова)
# ...

# Сохранение лучшей модели
# ...

-----

## **5. FastText embeddings**

Обучим модель FastText и сравним ее с Word2Vec.

- Обучите модель FastText.
- По аналогии с Word2Vec, получите векторы документов и обучите классификатор.
- Оцените качество на тестовой выборке.
- Продемонстрируйте преимущество FastText на OOV-словах (словах, отсутствующих в словаре).


In [ ]:
# ЗАДАНИЕ: Обучите модель FastText с параметрами по вашему выбору.
# fasttext_model = ...

# Векторизация и обучение классификатора
# ...

# ЗАДАНИЕ: Оцените модель на тестовой выборке (метрики и confusion matrix).
# ...

# Тест на OOV (out-of-vocabulary) словах и сравнение с Word2Vec
# ...

# Сохранение модели
# ...

-----

## **6. Отчеты и выводы**
### **6.1. Визуализация и анализ**

Сравним все полученные модели и визуализируем результаты.

- Подготовьте сводную таблицу и/или график со сравнительными метриками всех моделей.
- Визуализируйте эмбеддинги с помощью t-SNE.
- Проанализируйте ошибки моделей.


In [ ]:
# ЗАДАНИЕ: Создайте сравнительную таблицу и графики для метрик всех моделей.
# Постройте общую ROC-кривую.
# ...

# t-SNE визуализация эмбеддингов
# (Используйте эмбеддинги лучшей модели Word2Vec или FastText)
# ...

# Анализ ошибок
# (Найдите примеры текстов, на которых все модели ошибаются)
# ...

-----

## **6. Отчеты и выводы**
### **6.2. Подготовка отчетов**

Подготовьте итоговый отчет в формате Markdown.

- Проанализируйте ошибки моделей.
- Сформулируйте итоговые выводы (5-8 предложений): какой метод показал себя лучше и почему, в чем преимущества и недостатки каждого подхода, какие дальнейшие шаги по улучшению качества можно предпринять.

In [ ]:
# ЗАДАНИЕ: Сгенерируйте отчет, включающий описание данных, шаги предобработки,
# результаты моделей, сравнительный анализ, выводы и рекомендации.
report = f"""
# ОТЧЕТ ПО ДОМАШНЕМУ ЗАДАНИЮ

## 1. Описание данных
...

## 2. Предобработка
...

... (и так далее)
"""

# Сохранение отчета в формате markdown, вы можете использовать также pdf и docx
with open('reports/final_report.md', 'w', encoding='utf-8') as f:
    f.write(report)
print("Отчет сохранен в reports/final_report.md")

-----

## **Дополнительное задание по желанию**

За задание дополнительные баллы не ставятся. Оно выполняется по желанию студента.

In [ ]:
# Примеры бонусных заданий:
# 1. Ансамблирование моделей (усреднение вероятностей).
# 2. Кросс-валидация для более надежной оценки.
# 3. Использование предобученных эмбеддингов (например, из gensim-data).
# 4. Анализ интерпретируемости с LIME.
# 5. Загрузку Word2Vec модели из Семинара 1